# Connect to DuckDB

This cell establishes a connection to the DuckDB database file `sakila.duckdb` so we can query the data. The connection uses a relative path from the `notebooks/` folder.


In [30]:
import duckdb
import pandas as pd

# Connect to DuckDB (relative path from notebooks folder)
con = duckdb.connect("../data/sakila.duckdb")




### Preview Staging Tables

This cell shows all tables in the `staging` schema and previews the first 5 rows of the `actor` table.


In [ ]:
tables = con.execute("SHOW TABLES").fetchall()
staging_tables = con.execute(
    "SELECT table_name FROM information_schema.tables WHERE table_schema='staging'"
).fetchall()
print("Tables in staging schema:", staging_tables)

# first 5 rows
This code shows first 5 rows of the table actor where we can see 

In [ ]:

sample_data = con.execute("SELECT * FROM staging.actor LIMIT 5").fetchdf()
sample_data
